In [1]:
import pandas

import automol
import helpers
import ioformat

In [2]:
# 1. read in the non-redundantly expanded species and reaction lists
NREX_C67_SPC_PATH = 'data/species/02_species-nr-exp_nuig-c6-7.csv'
NREX_C67_RXN_PATH = 'data/reactions/04_reactions-nr-exp_nuig-c6-7.csv'
NREX_PYR_SPC_PATH = 'data/species/02_species-nr-exp_nuig-pyro.csv'
NREX_PYR_RXN_PATH = 'data/reactions/04_reactions-nr-exp_nuig-pyro.csv'

NREX_C67_SPC_DF = pandas.read_csv(NREX_C67_SPC_PATH, quotechar="'")
NREX_C67_RXN_DF = pandas.read_csv(NREX_C67_RXN_PATH, quotechar="'")
NREX_PYR_SPC_DF = pandas.read_csv(NREX_PYR_SPC_PATH, quotechar="'")
NREX_PYR_RXN_DF = pandas.read_csv(NREX_PYR_RXN_PATH, quotechar="'")

In [3]:
# 2. add missing non-canonical species enantiomers back into
#    non-redundant species list, adding columns containing
#    racemic ChIs and racemic CHEMKIN names
def add_missing_species_enantiomers(spc_df):
    row_dcts = []
    count = len(spc_df.index)
    for num, row0 in spc_df.iterrows():
        print(f'Row {num}/{count}')
        row_dct0 = row0.to_dict()
        name0 = row_dct0['name']
        chi0 = row_dct0['inchi']
        smi0 = row_dct0['smiles']
        # racemic name/ChI
        is_enant = automol.chi.is_enantiomer(chi0)
        namer = helpers.racemic_species_name(name0, chi0)
        chir = automol.chi.racemic(chi0)
        print(f'Species: {name0}')
        print(f' - ChI: {chi0}')
        print(f' - SMILES: {smi0}')
        row_dct0.update({
            'name': name0,
            'racem-name': namer,
            'racem-inchi': chir,
            'is_enant': is_enant,
        })
        row_dcts.append(row_dct0)
        if is_enant:
            print(f' - Species is chiral.')
            print(f' - Racemic name: {namer}')
            print(f' - Racemic ChI: {chir}')
            print(f' - Checking for missing enantiomer...')
            # name/ChI for other enantiomer
            name1 = helpers.reflect_species_name(name0, chi0)
            if name1 not in spc_df.index:
                chi1 = automol.chi.reflect(chi0)
                smi1 = automol.smiles.reflect(smi0)
                print(f' - Missing enantiomer: {name1}')
                print(f'   - ChI: {chi1}')
                print(f'   - SMILES: {smi1}')
                row_dct1 = row_dct0.copy()
                row_dct1.update({
                    'name': name1,
                    'inchi': chi1,
                    'smiles': smi1,
                    'is_enant': is_enant,
                })
                row_dcts.append(row_dct1)
    ex_spc_df = pandas.DataFrame.from_records(row_dcts)
    return ex_spc_df

In [4]:
print('Retrieving full NUIG-C6-7 species expansion from non-redundant list')
FUEX_C67_SPC_DF = add_missing_species_enantiomers(NREX_C67_SPC_DF)

Retrieving full NUIG-C6-7 species expansion from non-redundant list
Row 0/2584
Species: O
 - ChI: InChI=1S/O
 - SMILES: [O]
Row 1/2584
Species: O2
 - ChI: InChI=1S/O2/c1-2
 - SMILES: O=O
Row 2/2584
Species: H
 - ChI: InChI=1S/H
 - SMILES: [H]
Row 3/2584
Species: HO2
 - ChI: InChI=1S/HO2/c1-2/h1H
 - SMILES: O[O]
Row 4/2584
Species: OH
 - ChI: InChI=1S/HO/h1H
 - SMILES: [OH]
Row 5/2584
Species: H2
 - ChI: InChI=1S/H2/h1H
 - SMILES: [H][H]
Row 6/2584
Species: H2O
 - ChI: InChI=1S/H2O/h1H2
 - SMILES: O
Row 7/2584
Species: H2O2
 - ChI: InChI=1S/H2O2/c1-2/h1-2H
 - SMILES: OO
Row 8/2584
Species: CO
 - ChI: InChI=1S/CO/c1-2
 - SMILES: [C]=O
Row 9/2584
Species: CO2
 - ChI: InChI=1S/CO2/c2-1-3
 - SMILES: O=C=O
Row 10/2584
Species: HCO
 - ChI: InChI=1S/CHO/c1-2/h1H
 - SMILES: [CH]=O
Row 11/2584
Species: O2CHO
 - ChI: InChI=1S/CHO3/c2-1-4-3/h1H
 - SMILES: O=CO[O]
Row 12/2584
Species: CH2O
 - ChI: InChI=1S/CH2O/c1-2/h1H2
 - SMILES: C=O
Row 13/2584
Species: HO2CHO
 - ChI: InChI=1S/CH2O3/c2-1-4-3/h1,

In [5]:
print('Retrieving full NUIG-Pyro species expansion from non-redundant list')
FUEX_PYR_SPC_DF = add_missing_species_enantiomers(NREX_PYR_SPC_DF)

Retrieving full NUIG-Pyro species expansion from non-redundant list
Row 0/431
Species: H
 - ChI: InChI=1S/H
 - SMILES: [H]
Row 1/431
Species: H2
 - ChI: InChI=1S/H2/h1H
 - SMILES: [H][H]
Row 2/431
Species: C
 - ChI: InChI=1S/C
 - SMILES: [C]
Row 3/431
Species: CH
 - ChI: InChI=1S/CH/h1H
 - SMILES: [CH]
Row 4/431
Species: CH2
 - ChI: InChI=1S/CH2/h1H2
 - SMILES: [CH2]
Row 5/431
Species: CH2(S)
 - ChI: InChI=1S/CH2/h1H2
 - SMILES: [CH2]
Row 6/431
Species: CH3
 - ChI: InChI=1S/CH3/h1H3
 - SMILES: [CH3]
Row 7/431
Species: CH4
 - ChI: InChI=1S/CH4/h1H4
 - SMILES: C
Row 8/431
Species: C2
 - ChI: InChI=1S/C2/c1-2
 - SMILES: [C]#[C]
Row 9/431
Species: C2H
 - ChI: InChI=1S/C2H/c1-2/h1H
 - SMILES: C#[C]
Row 10/431
Species: C2H2
 - ChI: InChI=1S/C2H2/c1-2/h1-2H
 - SMILES: C#C
Row 11/431
Species: H2CC
 - ChI: InChI=1S/C2H2/c1-2/h1H2
 - SMILES: C=[C]
Row 12/431
Species: C2H3
 - ChI: InChI=1S/C2H3/c1-2/h1H,2H2
 - SMILES: [CH]=C
Row 13/431
Species: C2H4
 - ChI: InChI=1S/C2H4/c1-2/h1-2H2
 - SMILES: C=

In [6]:
# 2. add missing non-canonical reaction enantiomers back into
#    non-redundant reaction list, adding columns with racemic
#    CHEMKIN names
def add_missing_reaction_enantiomers(rxn_df, spc_df):
    """ Add missing enantiomers back into non-redundant raction list
    
        :param rxn_df: fully or non-redundantly expanded reaction list
        :param spc_df: fully expanded species list, with 'racem-name'
            and 'racem-inchi' columns added
    """
    row_dcts = []
    count = len(rxn_df.index)
    chi_dct = dict(zip(spc_df['name'], spc_df['inchi']))
    for num, row0 in rxn_df.iterrows():
        print(f'Row {num}/{count}')
        row_dct0 = row0.to_dict()
        name0 = row_dct0['name']
        print(f'Reaction: {name0}')
        namer = helpers.racemic_reaction_name(name0, chi_dct)
        name1 = helpers.reflect_reaction_name(name0, chi_dct)
        num_rct_enants = helpers.chiral_reactant_count(name0, chi_dct)
        row_dct0.update({
            'name': name0,
            'racem-name': namer,
            'num-reac-enants': num_rct_enants,
        })
        row_dcts.append(row_dct0)
        if name1 is not None:
            print(f' - Reaction is chiral.')
            print(f' - Racemic name: {namer}')
            if name1 not in rxn_df.index:
                print(f' - Missing enantiomer: {name1}')
                row_dct1 = row_dct0.copy()
                ts_amchi = row_dct0['ts_amchi_stereo']
                row_dct1.update({
                    'name': name1,
                    'num-reac-enants': num_rct_enants,
                    'ts_amchi_stereo': automol.chi.reflect(ts_amchi),
                })
                row_dcts.append(row_dct1)
    ex_rxn_df = pandas.DataFrame.from_records(row_dcts).set_index('name')
    ex_rxn_df.insert(0, 'racem-name', ex_rxn_df.pop('racem-name'))
    return ex_rxn_df

In [7]:
print('Retrieving full NUIG-C6-7 reaction expansion from non-redundant list')
FUEX_C67_RXN_DF = add_missing_reaction_enantiomers(
    NREX_C67_RXN_DF, FUEX_C67_SPC_DF)

Retrieving full NUIG-C6-7 reaction expansion from non-redundant list
Row 0/5990
Reaction: C6H11Q12-3-A0 = C6H11-2D1OOH-Z + HO2
 - Reaction is chiral.
 - Racemic name: C6H11Q12-3-Ar = C6H11-2D1OOH-Z + HO2
 - Missing enantiomer: C6H11Q12-3-A1 = C6H11-2D1OOH-Z + HO2
Row 1/5990
Reaction: C6H11Q12-3-A0 = C6H11-2D1OOH-E + HO2
 - Reaction is chiral.
 - Racemic name: C6H11Q12-3-Ar = C6H11-2D1OOH-E + HO2
 - Missing enantiomer: C6H11Q12-3-A1 = C6H11-2D1OOH-E + HO2
Row 2/5990
Reaction: C5H10-2-Z + CH3 = C5H92-5-Z + CH4
Row 3/5990
Reaction: C5H10-2-E + CH3 = C5H92-5-E + CH4
Row 4/5990
Reaction: C6KET31 = C6KET31O + OH
Row 5/5990
Reaction: C7H14-2-Z + OH = C7H132-4-ZZ + H2O
Row 6/5990
Reaction: C7H14-2-Z + OH = C7H132-4-ZE + H2O
Row 7/5990
Reaction: C7H14-2-E + OH = C7H132-4-EZ + H2O
Row 8/5990
Reaction: C7H14-2-E + OH = C7H132-4-EE + H2O
Row 9/5990
Reaction: C7H131-5O2-A0 = C7H121OOH5-6-A0
 - Reaction is chiral.
 - Racemic name: C7H131-5O2-Ar = C7H121OOH5-6-Ar
 - Missing enantiomer: C7H131-5O2-A1 

In [8]:
print('Retrieving full NUIG-Pyro reaction expansion from non-redundant list')
FUEX_PYR_RXN_DF = add_missing_reaction_enantiomers(
    NREX_PYR_RXN_DF, FUEX_PYR_SPC_DF)

Retrieving full NUIG-Pyro reaction expansion from non-redundant list
Row 0/1646
Reaction: C3H5-A + C2H5 = C2H4 + C3H6
Row 1/1646
Reaction: IC3H7 + H = C3H8
Row 2/1646
Reaction: C4H10 = SC4H9 + H
Row 3/1646
Reaction: C5H10-2-Z + CH3 = C5H92-5-Z + CH4
Row 4/1646
Reaction: C5H10-2-E + CH3 = C5H92-5-E + CH4
Row 5/1646
Reaction: IC5H12 = IC3H7 + C2H5
Row 6/1646
Reaction: C6H12-1 + H = C5H10-1 + CH3
Row 7/1646
Reaction: B13DE2M + H = AC5H9-C-Z
Row 8/1646
Reaction: B13DE2M + H = AC5H9-C-E
Row 9/1646
Reaction: NEC6 + H = NEC6-4 + H2
Row 10/1646
Reaction: IC6 + H = IC6-5 + H2
Row 11/1646
Reaction: B13DE2M + H = CC5H9-A-A0
 - Reaction is chiral.
 - Racemic name: B13DE2M + H = CC5H9-A-Ar
 - Missing enantiomer: B13DE2M + H = CC5H9-A-A1
Row 12/1646
Reaction: C6H10D14-Z + H = C6H111-5
Row 13/1646
Reaction: C6H10D14-E + H = C6H111-5
Row 14/1646
Reaction: IC6D2 + H = IC6D2-5 + H2
Row 15/1646
Reaction: C6H12-1 + H = C6H111-2 + H2
Row 16/1646
Reaction: XC6D1 = AC5H9-C-Z + CH3
Row 17/1646
Reaction: XC6D1

In [9]:
# 4. construct a complete, non-redundant model by lumping enantiomers
def lump_enantiomers(rxn_df, spc_df):
    """ Lump species and reactions, assuming a racemic mixture
    """
    # First, lump species. This list is equivalent to the non-redundant species list,
    # with names replaced by racemic names
    lu_spc_df = spc_df.groupby('racem-name', sort=False).apply(lambda g: g.iloc[0])
    lu_spc_df.rename({'name': 'stereo-name'}, axis=1, inplace=True)
    lu_spc_df.index.name = 'name'
    lu_spc_df.pop('racem-name')
    lu_spc_df.reset_index(inplace=True)

    # Second, lump reactions. This is *not* equivalent to the non-redundant reaction list
    # (a.) Convert serialized parameter strings into objects
    rxn_df = helpers.reactions_with_params_objects(rxn_df)

    # (b.) Lump products, adding rate constants
    def _lump_reactions(grp_df):
        row = grp_df.iloc[0]
        row['stereo-name'] = grp_df.index[0]
        objs = grp_df['params']
        nreac = row['num-reac-enants']
        row['params'] = sum(objs) / (2 ** nreac)  # __radd__ is overloaded
        return row

    lu_rxn_df = rxn_df.groupby('racem-name', sort=False).apply(_lump_reactions)
    lu_rxn_df.index.name = 'name'
    lu_rxn_df.pop('racem-name')
    lu_rxn_df.insert(0, 'stereo-name', lu_rxn_df.pop('stereo-name'))
    lu_rxn_df.reset_index(inplace=True)
    return lu_rxn_df, lu_spc_df


LUMP_C67_RXN_DF, LUMP_C67_SPC_DF = lump_enantiomers(FUEX_C67_RXN_DF, FUEX_C67_SPC_DF)
LUMP_PYR_RXN_DF, LUMP_PYR_SPC_DF = lump_enantiomers(FUEX_PYR_RXN_DF, FUEX_PYR_SPC_DF)

LUMP_C67_RXN_PATH = 'data/reactions/06_reactions-lumped_nuig-c6-7.csv'
LUMP_C67_RXN_DF.to_csv(LUMP_C67_RXN_PATH, quotechar="'")
LUMP_PYR_RXN_PATH = 'data/reactions/06_reactions-lumped_nuig-pyro.csv'
LUMP_PYR_RXN_DF.to_csv(LUMP_PYR_RXN_PATH, quotechar="'")

/tmp/ipykernel_9985/2872968592.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lu_spc_df = spc_df.groupby('racem-name', sort=False).apply(lambda g: g.iloc[0])
/tmp/ipykernel_9985/2872968592.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lu_rxn_df = rxn_df.groupby('racem-name', sort=False).apply(_lump_reactions)
/tmp/ipykernel_9985/2872968592.py:7: DeprecationWarning: DataFrameGroupBy.apply operate

In [10]:
# 5. generate reaction parameters dictionaries
LUMP_C67_RXN_DCT = helpers.mechanism_dict(LUMP_C67_RXN_DF)
LUMP_PYR_RXN_DCT = helpers.mechanism_dict(LUMP_PYR_RXN_DF)

In [11]:
# 6. restrict the species dataframes based on the reaction lists
LUMP_C67_SPC_DF = helpers.species_for_mechanism(LUMP_C67_SPC_DF, LUMP_C67_RXN_DCT)
LUMP_PYR_SPC_DF = helpers.species_for_mechanism(LUMP_PYR_SPC_DF, LUMP_PYR_RXN_DCT)

In [12]:
# 7. write the original and fully expanded submechanisms
LUMP_C67_SPC_PATH = 'data/mechanisms/06_nr-exp_nuig-c6-7.csv'
LUMP_C67_RXN_PATH = 'data/mechanisms/06_nr-exp_nuig-c6-7.txt'
LUMP_PYR_SPC_PATH = 'data/mechanisms/06_nr-exp_nuig-pyro.csv'
LUMP_PYR_RXN_PATH = 'data/mechanisms/06_nr-exp_nuig-pyro.txt'

# a. write the species files
LUMP_C67_SPC_DF.to_csv(LUMP_C67_SPC_PATH, quotechar="'", index=False)
LUMP_PYR_SPC_DF.to_csv(LUMP_PYR_SPC_PATH, quotechar="'", index=False)

# b. write the reaction/mechanism files
LUMP_C67_MECH_STR = helpers.mechanism_string(LUMP_C67_RXN_DCT, LUMP_C67_SPC_DF)
LUMP_PYR_MECH_STR = helpers.mechanism_string(LUMP_PYR_RXN_DCT, LUMP_PYR_SPC_DF)

ioformat.pathtools.write_file(LUMP_C67_MECH_STR, '.', LUMP_C67_RXN_PATH)
ioformat.pathtools.write_file(LUMP_PYR_MECH_STR, '.', LUMP_PYR_RXN_PATH)